In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py 

PATH = '/storage/home/adz6/group/project'
VNADATAPATH = os.path.join(PATH, 'vna_data')
DAQDATAPATH = os.path.join(PATH, 'daq_data')
MOTHPATH = os.path.join(PATH, 'moth')
MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
PLOTPATH = os.path.join(PATH, 'plots/moth')
RAWSIMDATAPATH = os.path.join(PATH, 'sim_data/datasets')

def LoadData(path2data, nskip_start=10240, remove_mean=True, convert_to_volts=True):
    
    antenna_angles = []
    nantenna = len(os.listdir(path2data))
    for iantenna, antenna in enumerate(os.listdir(path2data)):
        antenna_angles.append(int(antenna))
        
        for ichannel, channel in enumerate(os.listdir(os.path.join(path2data, antenna))):
            
            signal_array = LoadArray(os.path.join(path2data, antenna, channel), nskip_start=nskip_start)
            if remove_mean:
                signal_array = signal_array - np.mean(signal_array)
            
            if iantenna == 0 and ichannel == 0:
                antenna_data = np.zeros((nantenna, signal_array.size))
                lo_data = np.zeros((nantenna, signal_array.size))
            #print(channel)
            if channel == 'wave0.txt':
                antenna_data[iantenna, :] = signal_array
            elif channel == 'wave1.txt':
                lo_data[iantenna, :] = signal_array
                
    antenna_sort_inds = np.argsort(antenna_angles)
    
    antenna_angles = -1 * np.sort(antenna_angles)
    
    antenna_data = scipy.signal.hilbert(antenna_data[antenna_sort_inds, :])
    lo_data = scipy.signal.hilbert(lo_data[antenna_sort_inds, :])
    
    if convert_to_volts:
        antenna_data *= 0.5 / 2**16
        lo_data *= 5 / 2**16
    
    return {'angle': antenna_angles, 'antenna': antenna_data, 'lo': lo_data}
    
    
def LoadArray(path2array, nskip_start=10240):
    
    signal_list = []
    with open(path2array, 'r') as infile:
            reader = csv.reader(infile)
            for i, item in enumerate(reader):
                if i > 6:
                    signal_list.append(int(item[0]))
    return np.array(signal_list)[nskip_start:]

def SumData(array_data, xpos, ypos, r=0.1, f=25.8e9, antispiral=False):
    
    xx, yy = np.meshgrid(xpos, ypos)
    xx, yy = np.expand_dims(xx, axis=0).repeat(array_data['angle'].size, axis=0), np.expand_dims(yy, axis=0).repeat(array_data['angle'].size, axis=0)

    xantenna = (r * np.cos(np.radians(-1 * array_data['angle']))).reshape((array_data['angle'].size, 1, 1)).repeat(xpos.size, axis=1).repeat(xpos.size, axis=2)
    yantenna = (r * np.sin(np.radians(-1 * array_data['angle']))).reshape((array_data['angle'].size, 1, 1)).repeat(ypos.size, axis=1).repeat(ypos.size, axis=2)

    xdist = xantenna - xx
    ydist = yantenna - yy
    
    dist_grid = np.sqrt(xdist**2 + ydist**2)
    
    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    if antispiral:
        antispiral_phase = np.exp(-1j * np.radians( 1 * array_data['angle'])).reshape((array_data['angle'].size, 1)).repeat(array_data['antenna'].shape[-1], axis=-1)
        return np.matmul(beamforming_phases.T, antispiral_phase * array_data['antenna'])
    else:
        return np.matmul(beamforming_phases.T, array_data['antenna'])
    
def SumDataLocust(array_data, xpos, ypos, r=0.1, f=25.8e9, antispiral=False, nch=60):
    
    angles = np.arange(0, nch, 1) * 360 // nch
    xx, yy = np.meshgrid(xpos, ypos)
    xx, yy = np.expand_dims(xx, axis=0).repeat(angles.size, axis=0), np.expand_dims(yy, axis=0).repeat(angles.size, axis=0)

    xantenna = (r * np.cos(np.radians(-1 * angles))).reshape((angles.size, 1, 1)).repeat(xpos.size, axis=1).repeat(xpos.size, axis=2)
    yantenna = (r * np.sin(np.radians(-1 * angles))).reshape((angles.size, 1, 1)).repeat(ypos.size, axis=1).repeat(ypos.size, axis=2)

    xdist = xantenna - xx
    ydist = yantenna - yy
    
    antispiral_phase_grid = np.degrees(np.arctan2(-ydist, xdist))
    
    dist_grid = np.sqrt(xdist**2 + ydist**2)
    
    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    #print(beamforming_phases.shape)
    
    if antispiral:
        antispiral_phase = np.exp(-1j * np.radians( 1 * angles)).reshape((angles.size, 1)).repeat(array_data.shape[-1], axis=-1)
        return np.matmul((beamforming_phases + .T, antispiral_phase * array_data)
    else:
        return np.matmul(beamforming_phases.T, array_data)

def CorrectLOPhase(array_data):
    
    lo_fft = np.fft.fft(array_data['lo'])
    
    phase_at_max = np.diag(np.angle(lo_fft[:, np.argmax(abs(lo_fft), axis=-1)]))
    
    phase_diff = phase_at_max - phase_at_max[0]
    
    correction = np.exp(-1j * phase_diff).reshape((*phase_diff.shape, 1)).repeat(lo_fft.shape[-1], axis=-1)
    
    array_data['antenna'] *= correction
    array_data['lo'] *= correction

def GetPhase(array):
    data_fft = np.fft.fft(array, axis=-1)
    #print(data_fft.shape)
    max_inds = np.argmax(abs(data_fft[:, 1, :]), axis=-1)
    phase_array = np.diag(np.angle(data_fft[:, 1, max_inds]))
    
    return phase_array

def PhaseShiftLocustData(array_data, new_x, new_y, r=0.1, f=25.8e9, nch=60):
    
    angles = np.arange(0, nch, 1) * 360 // nch
    #r_shift = np.sqrt((new_x) ** 2 + (new_y) ** 2)
    
    xantenna = (r * np.cos(np.radians(-1 * angles)))
    yantenna = (r * np.sin(np.radians(-1 * angles)))
    
    dist_old = np.sqrt(xantenna ** 2 + yantenna ** 2)

    dist_new = np.sqrt((xantenna - new_x) ** 2 +  (yantenna - new_y) ** 2)
    #print(dist_old - dist_new)
    

    phase_shift = np.exp(1j * 2 * np.pi * f * (dist_old - dist_new) / 3e8).reshape((nch, 1)).repeat(array_data.shape[-1], axis=-1)
    
    #print(phase_shift)
    
    array_data = phase_shift * array_data
    
    return array_data
    
    
    
    

In [ ]:
os.listdir(os.path.join(RAWSIMDATAPATH, '210715_df_3timeslice_unsummed_data'))

In [ ]:
path2file = os.path.join(RAWSIMDATAPATH, '210715_df_3timeslice_unsummed_data','210629_df1.h5' )

In [ ]:
h5file = h5py.File(path2file, 'r')

In [ ]:
xpos = np.linspace(-0.03, 0.03, 101)
ypos = np.linspace(-0.03, 0.03, 101)

signal_data = h5file['signal']['0'][:, 0:8192]

signal_data = PhaseShiftLocustData(signal_data, 0.01, 0.0)

summed_data_anti = SumDataLocust(signal_data, xpos, ypos, antispiral=True)
summed_data = SumDataLocust(signal_data, xpos, ypos, antispiral=False)

#print(np.degrees(np.diag(np.angle(np.fft.fft(array_data['antenna'])[:, np.argmax(abs(np.fft.fft(array_data['antenna'])), axis=-1)]))))
#measured_phase = np.degrees(np.unwrap(np.diag(np.angle(np.fft.fft(array_data['antenna'])[:, np.argmax(abs(np.fft.fft(array_data['antenna'])), axis=-1)]))))

In [ ]:
summed_data_anti_fft = np.fft.fft(summed_data_anti, axis=-1) / summed_data_anti.shape[-1]
summed_data_fft = np.fft.fft(summed_data, axis=-1) / summed_data.shape[-1]
#peak_ind = np.argmax(abs(np.fft.fft(array_data['lo'][0, :])))
#print(np.argmax(abs(np.fft.fft(array_data['lo'][0, :]))))

image_anti = abs(summed_data_anti_fft)[:, :, 167]
image = abs(summed_data_fft)[:, :, 167]

In [ ]:
figname = '210810_locust_electron_beamforming.png'

max_index = np.unravel_index(np.argmax(image_anti), (101, 101))
print(f'(x = {xpos[max_index[0]]}, y = {ypos[max_index[1]]})')

sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow((image_anti**2).T, extent=(-30, 30, 30, -30), cmap=cmap)
plt.colorbar(img, label = r'$V^2$')

ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_title('Reconstructed Electron Position')
#plt.plot(xpos[max_index[0]], ypos[max_index[1]], 'r.')

plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, figname))

In [ ]:
figname = '210810_locust_electron_beamforming_no_correction.png'

max_index = np.unravel_index(np.argmax(image_anti), (101, 101))
print(f'(x = {xpos[max_index[0]]}, y = {ypos[max_index[1]]})')

sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(image**2, extent=(-30, 30, 30, -30), cmap=cmap)
plt.colorbar(img, label = r'$V^2$')

ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_title('Reconstructed Electron Position')
#plt.plot(xpos[max_index[0]], ypos[max_index[1]], 'r.')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, figname))

# augment locust signal to appear like it is off-axis and save the beamforming result

In [ ]:
xpos = np.linspace(-0.03, 0.03, 101)
ypos = np.linspace(-0.03, 0.03, 101)

images  = np.zeros((31, 101, 101))




for i, distance in enumerate(np.arange(0, 31, 1) / 2000):

    print(distance)

    signal_data = h5file['signal']['0'][:, 0:8192]

    signal_data = PhaseShiftLocustData(signal_data, distance, 0.0)
    
    summed_data_anti = SumDataLocust(signal_data, xpos, ypos, antispiral=False)
    
    summed_data_anti_fft = np.fft.fft(summed_data_anti, axis=-1) / summed_data_anti.shape[-1]

    image_anti = abs(summed_data_anti_fft)[:, :, 167]
    
    print(image_anti.shape)
    
    images[i, :, :] = image_anti ** 2
    
    
np.save(os.path.join(PATH, 'results/moth/211019_augmented_off_axis_beamforming_range_locust_no_antispiral'), images)
    

In [ ]:
np.degrees(2 * np.pi * 1e-3 * 25.8e9 / 3e8)

In [ ]:
data_fft = np.fft.fft(data_array, axis=-1)
#print(np.argmax(abs(data_fft[:, 1, :]), axis=-1))
max_inds = np.argmax(abs(data_fft[:, 1, :]), axis=-1)
phase_array = np.diag(np.angle(data_fft[:, 1, max_inds]))

phase_diff_array = phase_array - phase_array[0]


lo_data = data_array[:, 1, :]
lo_data_corrected = np.exp(-1j * phase_diff_array.reshape((*phase_diff_array.shape, 1)).repeat(lo_data.shape[-1], axis=-1)) * lo_data

antenna_data = data_array[:, 0, :]
antenna_data_corrected = np.exp(-1j * phase_diff_array.reshape((*phase_diff_array.shape, 1)).repeat(lo_data.shape[-1], axis=-1)) * antenna_data

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(lo_data_corrected[0, :], color = 'tab:blue')
ax1.plot(lo_data_corrected[3, :], color = 'tab:red')

ax1.set_xlim(0, 100)

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
#ax1.plot(antenna_data_corrected[0, :], color = 'tab:blue')
#ax1.plot(antenna_data_corrected[3, :], color = 'tab:red')

ax1.plot(np.mean(antenna_data_corrected, axis=0), color = 'tab:orange')
ax1.set_xlim(0, 100)

In [ ]:
#plt.plot(data_array[0, 0, :])
fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(data_array[0, 0, :], color = 'tab:blue')
ax1.plot(data_array[3, 0, :], color = 'tab:red')

ax2 = ax1.twinx()
ax2.plot(data_array[0, 1, :], color = 'tab:orange')
#ax2.plot(data_array[3, 1, :], color = 'tab:green')

ax2.plot(data_array[3, 1, :], color = 'tab:green')

ax1.set_xlim(0, 100)

In [ ]:
fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(data_array[0, 0, :], color = 'tab:blue')
#ax1.plot(data_array[3, 0, :], color = 'tab:red')

#ax2 = ax1.twinx()
#ax2.plot(data_array[0, 1, :], color = 'tab:orange')
#ax2.plot(data_array[3, 1, :], color = 'tab:green')

In [ ]:
fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(abs(np.fft.rfft(data_array[0, 1, :]-np.mean(data_array[0, 1, :]))), color = 'tab:blue')

In [ ]:
index = np.argmax(abs(np.fft.rfft(data_array[0, 1, :]-np.mean(data_array[0, 1, :]))))
print(np.angle(np.fft.rfft(data_array[0, 1, :]-np.mean(data_array[0, 1, :])))[index])
print(np.angle(np.fft.rfft(data_array[3, 1, :]-np.mean(data_array[3, 1, :])))[index])

In [ ]:
phase_diff = np.angle(np.fft.rfft(data_array[0, 1, :]-np.mean(data_array[0, 1, :])))[index] - np.angle(np.fft.rfft(data_array[1, 1, :]-np.mean(data_array[1, 1, :])))[index]
print(phase_diff)

In [ ]:
signal1 = scipy.signal.hilbert(data_array[0, 1, :] - np.mean(data_array[0, 1, :]))
signal2 = scipy.signal.hilbert(data_array[1, 1, :] - np.mean(data_array[1, 1, :]))

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(signal1.real, color = 'tab:blue')
ax1.plot(signal2.real, color = 'tab:red')

ax1.set_xlim(0, 100)

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(signal1.real, color = 'tab:blue')
ax1.plot((signal2 * np.exp(1j * phase_diff)).real, color = 'tab:red')

ax1.set_xlim(0, 100)

In [ ]:
signal1 = scipy.signal.hilbert(data_array[0, 0, :] - np.mean(data_array[0, 0, :]))
signal2 = scipy.signal.hilbert(data_array[1, 0, :] - np.mean(data_array[1, 0, :]))

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
ax1.plot(signal1.real, color = 'tab:blue')
ax1.plot(signal2.real, color = 'tab:red')

ax1.set_xlim(0, 100)

fig = plt.figure(figsize=(13,8))

ax1 = fig.add_subplot(1,1,1)
#ax1.plot(signal1.real, color = 'tab:blue')
#ax1.plot((signal2 * np.exp(1j * phase_diff)).real, color = 'tab:red')
ax1.plot(signal1.real + (signal2 * np.exp(1j * phase_diff)).real, color = 'tab:green')
ax1.set_xlim(0, 100)